In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import sys
sys.path.insert(1, '../Config')
from lablib import LS_fit

# Prima parte

## Circuito RC

Abbiamo effettuato le misurazioni con il cursore dell'oscilloscopio. \
**Resistenza interna oscilloscopio** = 50 $\Omega$

Intrvallo tensione (0.5,-0.5) V \
Capacità = 10 nF \
resistenza = 67.1 $k\Omega$ \
frequenza = 200 Hz \
incertezza su oscilloscopio (precisione strumento) = 8 mV


In [ ]:
# Primo plot dei dati
dati1 = pd.read_csv("Dati1.csv")

tempi1 = dati1["tempo(carica)"]
tens1 = dati1["tensione(carica1)"]

# plt.errorbar(tempi1, tens1, [8 for i in tens1], fmt = '.')
# plt.show()

In [ ]:
# prova fit dei dati con libreria
def mod_exp(x, A, T, C):
    return A*np.exp(-x/T) + C

tempi1_µs = [i*1e-3 for i in tempi1]
tens1_mv = [i*1e-3 for i in tens1]
tens1_err = [.008 for i in tens1]

par1, val1, ers1, pval1, *other = LS_fit(tempi1_µs, tens1_mv, tens1_err, mod_exp, disp=1, A=1, T=1, C=0) # type: ignore

for p, v, e in zip(par1, val1, ers1):
    print(f"{p} = {v:.3f} ± {e:.3f}")

In [ ]:
QC, tau_1, c_0 = val1
R_1 = 67.1e3 # Ohm

C = tau_1/R_1
print(f"Capacità calcolata = {C:.3g} F")

## Circuito RL

induttanza ~ 10 mH \
resistenza = 1 $\text{k}\Omega$ \
frequenza = 700 Hz \
range di tensione = (-1,1) V \
precisione oscilloscopio = 20 mV

In [ ]:
# plot dei dati

dati2 = pd.read_csv("Dati2.csv")
tempi2 = dati2["tempo"]
tens2 = dati2["tensione(carica)"]

# plt.errorbar(tempi2, tens2, [20 for i in tens2])
# plt.show()

In [ ]:
# Fit circuito RL

tempi2_ms = [i*1e-3 for i in tempi2]
tens2_mv = [i*1e-3 for i in tens2]
tens2_err = [.02 for i in tens2]

par2, val2, ers2, pval2, *other2 = LS_fit(tempi2_ms, tens2_mv, tens2_err, mod_exp, A=1, T=1, C=1)

for p, v, e in zip(par2, val2, ers2):
    print(f"{p} = {v:.3g} ± {e:.3g}")

In [ ]:
Va, tau_2, c_1 = val2
R_2 = 1e3 # ohm

L = R_2 * tau_2
print(f"Induttanza misurata = {L:.3g} H")

 # Seconda parte

## Circuito RLC

Resistenza = 300 $\Omega$ \
Induttanza e capacità uguali a prima \
Intervallo tensione (1,0)

Abbiamo stimato le costanti del sistema prima di prendere i dati:
* $\omega_0$ = 100 kHz
* $f_0 = \frac{\omega_0}{2\pi}$ = 16kHz 

Ci siamo resi presto conto che questi risultati sono insensati \
Frequenza effettivamente usata = 300 Hz


Per smorzamento critico abbiamo usato Resistenza = 3900 $\Omega$ \
Per sovrasmorzamento abbiamo usato Resistenza = 10_000 $\Omega$

### Caso sottosmorzato

In [ ]:
#plot dati RLC sottosmorzato

dati3 = pd.read_csv("Dati3.csv")
tempi3=dati3["tempo"]
tensione3=dati3["tensione"]

tempi3_s = [i*1e-3 for i in tempi3]
tensione3_v = [i*1e-3 for i in tensione3]

tens3_err = [8 for i in tensione3]
tens3_err_s = [.008 for i in tensione3]

plt.errorbar(tempi3, tensione3_v, tens3_err_s, fmt = '.',label="Data points RLC")
plt.title("Plot Circuito RLC sottosmorzato")
plt.xlabel("Tempo [s]")
plt.ylabel("Tensione [V]")
plt.legend()
plt.grid()
plt.show()

In [ ]:
# fit dei dati nel caso sottosmorzato

R_3 = R_1
C_3 = C
L_3 = L

omega_0_calc = 1/np.sqrt(L_3*C_3)
gamma_calc_1 = R_3/(2*L_3)
omega_calc_1 = np.sqrt(gamma_calc_1**2 - omega_0_calc**2)

def mod_ssm(x, I0, gamma, omega):
    return I0 * np.exp(-gamma*x)*np.sin(omega*x)

par3, val3, ers3, pval3, *other3 = LS_fit(tempi3_s, tensione3_v, tens3_err_s, mod_ssm, 
                                          I0 = 0.113, 
                                          gamma = 3000., 
                                          omega = 45360.) #non so perche' proprio questi valori

for p, v, e in zip(par3, val3, ers3):
    print(f"{p} = {v:.3g} ± {e:.3g}")

### Caso Smorzamento Critico 

In [ ]:
#plot dati RLC sottosmorzato
#resistenza usata R = 3900 omh

Dati3 = pd.read_csv("Dati3.csv")
tempo4 = Dati3["tempo2"].dropna()
tensione4 = Dati3["tensione2"].dropna()
tensione4_err = [8 for i in tensione4]

#trasporto le grandezze in SI
tempo4_s = [i*1e-3 for i in tempo4]
tensione4_v = [i*1e-3 for i in tensione4]
tensione4_err_v = [1e-3 for i in tensione4_err]

plt.errorbar(tempo4_s,tensione4_v,tensione4_err_v, fmt = '.',label="Data points RLC")
plt.title("Plot Circuito RLC smorzamento critico")
plt.xlabel("Tempo [s]")
plt.ylabel("Tensione [V]")
plt.legend()
plt.grid()
plt.show()

In [ ]:
#modello criticamente smorzato
def mod_csm(x, I0, gamma):
    return I0 * x * np.exp(-gamma*x)
#fit smorzamento critico
par4, val4, ers4, pval4, *other4 = LS_fit(tempo4_s, tensione4_v, tensione4_err_v, mod_csm, 
                                          I0 = 76000, 
                                          gamma = 39000)

for p, v, e in zip(par4, val4, ers4):
    print(f"{p} = {v:.3g} ± {e:.3g}")

### Caso sovrasmorzato

In [ ]:
#plot dati RLC sovrasmorzato 
#resistenza utilizzato R= 10.000 ohm 

Dati5 = pd.read_csv("Dati3.csv")
tempo5= Dati5["tempo3"].dropna()
tensione5= Dati5["tensione3"].dropna()
tensione5_err = [8 for i in tensione5]

#trasporto le grandezze in SI
tempo5_s = [i*1e-3 for i in tempo5]
tensione5_v = [i*1e-3 for i in tensione5]
tensione5_err_v = [1e-3 for i in tensione5_err]

plt.errorbar(tempo5_s,tensione5_v, tensione5_err_v, fmt = '.',label="Data points RLC")
plt.title("Plot Circuito RLC sovrasmorzato")
plt.xlabel("Tempo [s]")
plt.ylabel("Tensione [V]")
plt.legend()
plt.grid()
plt.show()

In [ ]:
# modello sovrasmorzato
def mod_svsm(x, I0, gamma, omega):
    return I0 * np.exp(-gamma*x) * (np.exp(omega*x) - np.exp(-omega*x))

#fit sovrasmorzamento
par5, val5, ers5, pval5, *other5 = LS_fit(tempo5_s, tensione5_v, tensione5_err_v, mod_svsm, 
                                          I0 = 1.15, 
                                          gamma = 100000, 
                                          omega = 45360) #non so perche' proprio questi valori

for p, v, e in zip(par5, val5, ers5):
    print(f"{p} = {v:.3g} ± {e:.3g}")